In [112]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
%matplotlib inline

train_data = pd.read_csv("train.csv")


In [18]:
def split_train_test(data, test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]
    
    

In [14]:
train_data.Age = train_data.Age.fillna(train_data.Age.mean())
train_data.Embarked = train_data.Embarked.fillna(2.0)
train_data = train_data.replace('male', 0).replace('female', 1).replace('C', 0).replace('Q', 1).replace('S', 2)
train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
train_data.dropna()

train_set, test_set = split_train_test(train_data, 0.2)



In [19]:
train_set.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,713.000000,713.000000,713.000000,713.000000,713.000000,713.000000,713.000000,713.000000
mean,0.380084,2.295933,0.347826,29.719489,0.516129,0.384292,33.044494,1.553997
std,0.485748,0.838470,0.476615,12.910690,1.048048,0.806142,52.197929,0.780891
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,23.000000,0.000000,0.000000,7.925000,1.000000
50%,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,14.454200,2.000000
75%,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.275000,2.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,5.000000,512.329200,2.000000


In [20]:
test_set.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,0.398876,2.359551,0.370787,29.617518,0.550562,0.370787,28.838342,1.466292
std,0.491049,0.826772,0.484378,13.398453,1.301987,0.807899,37.982056,0.831233
min,0.000000,1.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,1.000000
50%,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,14.479150,2.000000
75%,1.000000,3.000000,1.000000,33.750000,1.000000,0.000000,29.021875,2.000000
max,1.000000,3.000000,1.000000,66.000000,8.000000,6.000000,263.000000,2.000000


In [16]:
test_set



,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
873,0,3,0,47.000000,0,0,9.0000,2.0
515,0,1,0,47.000000,0,0,34.0208,2.0
535,1,2,1,7.000000,0,2,26.2500,2.0
85,1,3,1,33.000000,3,0,15.8500,2.0
707,1,1,0,42.000000,0,0,26.2875,2.0
63,0,3,0,4.000000,3,2,27.9000,2.0
307,1,1,1,17.000000,1,0,108.9000,0.0
587,1,1,0,60.000000,1,1,79.2000,0.0
863,0,3,1,29.699118,8,2,69.5500,2.0
177,0,1,1,50.000000,0,0,28.7125,0.0


In [21]:

sample_x = train_set.drop(['Survived'], axis=1)
sample_y = train_set.drop(["Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked", "Sex"], axis=1)

In [22]:
rf = RandomForestClassifier(min_samples_leaf=1, random_state=0)
rf.fit(sample_x, sample_y.values.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [78]:
test_sample_x = test_set.drop(['Survived'], axis=1)
test_sample_y = test_set.drop(["Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked", "Sex"], axis=1)

rf_result = rf.predict(test_sample_x)

print("Result from Random Forrest")
print(np.sum(rf_result==test_sample_y['Survived']))
print(np.sum(rf_result == test_sample_y['Survived']) / len(result))

Result from Random Forrest
141
0.7921348314606742


In [70]:
dt = DecisionTreeRegressor()
dt.fit(sample_x, sample_y.values.ravel())

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [79]:
test_sample_x = test_set.drop(['Survived'], axis=1)
test_sample_y = test_set.drop(["Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked", "Sex"], axis=1)

dt_result = dt.predict(test_sample_x)

print("Result from Decision Tree")
print(np.sum(dt_result==test_sample_y['Survived']))
print(np.sum(dt_result == test_sample_y['Survived']) / len(result))

Result from Decision Tree
123
0.6910112359550562


In [69]:
sv = svm.SVC()
sv.fit(sample_x, sample_y.values.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [80]:
test_sample_x = test_set.drop(['Survived'], axis=1)
test_sample_y = test_set.drop(["Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked", "Sex"], axis=1)

sv_result = sv.predict(test_sample_x)

print("Result from svm")
print(np.sum(sv_result==test_sample_y['Survived']))
print(np.sum(sv_result == test_sample_y['Survived']) / len(result))

Result from svm
125
0.702247191011236


In [102]:
rf_mse = mean_squared_error(test_sample_y, rf_result)
rf_rmse = np.sqrt(rf_mse)
rf_rmse

0.45592232730951643

In [103]:
dt_mse = mean_squared_error(test_sample_y, dt_result)
dt_rmse = np.sqrt(dt_mse)
dt_rmse

0.5165989964813013

In [104]:
sv_mse = mean_squared_error(test_sample_y, sv_result)
sv_rmse = np.sqrt(sv_mse)
sv_rmse

0.5456673061387901

In [96]:
scores = cross_val_score(dt, sample_x, sample_y, scoring="neg_mean_squared_error", cv = 10)
tree_rmse_scores = np.sqrt(-scores)


In [106]:
scores = cross_val_score(rf, sample_x, sample_y.values.ravel(), scoring="neg_mean_squared_error", cv = 10)
rf_rmse_scores = np.sqrt(-scores)

In [107]:
scores = cross_val_score(sv, sample_x, sample_y.values.ravel(), scoring="neg_mean_squared_error", cv = 10)
sv_rmse_scores = np.sqrt(-scores)

In [94]:
def display_scores(scores):
    print("Scores", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [95]:
display_scores(tree_rmse_scores)

Scores [0.38722514 0.51547604 0.47900951 0.39841956 0.45995113 0.4933069
 0.41173    0.39432816 0.55048794 0.48593782]
Mean: 0.4575872202088034
Standard deviation: 0.05392776795504263


In [100]:
display_scores(rf_rmse_scores)

Scores [0.40544243 0.5        0.37529331 0.48932261 0.37529331 0.45963856
 0.3560345  0.33567254 0.54385168 0.50350881]
Mean: 0.4344057758946008
Standard deviation: 0.06958918084107268


In [108]:
display_scores(sv_rmse_scores)

Scores [0.54897164 0.60092521 0.50350881 0.5814019  0.5814019  0.56916049
 0.45963856 0.54385168 0.61666984 0.54385168]
Mean: 0.5549381712178463
Standard deviation: 0.04420062634012379


In [185]:
# random forest
param_grid = [
        {
            'n_estimators': [3, 10, 15, 20, 25, 30], 
            'max_features': [2, 3, 4, 5, 6, 7],
            'max_depth': [3, 5, 10, 15, 20],
    },
    {
        'bootstrap': [False], 
        'n_estimators': [3, 10, 15, 20, 25, 30], 
        'max_features': [2, 3, 4, 5, 6, 7],
        'max_depth': [3, 5, 10, 15, 20],
    },
]

rf_grid_search = GridSearchCV(rf, param_grid, cv = 10,scoring = 'neg_mean_squared_error')

rf_grid_search.fit(sample_x, sample_y.values.ravel())

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 15, 20, 25, 30], 'max_features': [2, 3, 4, 5, 6, 7], 'max_depth': [3, 5, 10, 15, 20]}, {'bootstrap': [False], 'n_estimators': [3, 10, 15, 20, 25, 30], 'max_features': [2, 3, 4, 5, 6, 7], 'max_depth': [3, 5, 10, 15, 20]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [186]:
rf_grid_search.best_params_

{'max_depth': 10, 'max_features': 6, 'n_estimators': 20}

In [187]:
rf_grid_result = grid_search.predict(test_sample_x)

print("Result from grid seasrched random forest")
print(np.sum(rf_grid_result==test_sample_y['Survived']))
print(np.sum(rf_grid_result == test_sample_y['Survived']) / len(result))

Result from grid seasrched random forest
143
0.8033707865168539


In [179]:
# Decision Tree
param_grid = [
        {
            'max_depth': [3, 5, 10, 15, 20],
            'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 100, 1000],
            'min_samples_leaf': [1, 10, 100],
            'max_features': [2, 3, 4, 5, 6, 7],
    },
]

dt_grid_search = GridSearchCV(dt, param_grid, cv = 10)

dt_grid_search.fit(sample_x, sample_y.values.ravel())

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [3, 5, 10, 15, 20], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 100, 1000], 'min_samples_leaf': [1, 10, 100], 'max_features': [2, 3, 4, 5, 6, 7]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [180]:
dt_grid_search.best_params_

{'max_depth': 5,
 'max_features': 5,
 'min_samples_leaf': 10,
 'min_samples_split': 6}

In [181]:
dt_grid_result = dt_grid_search.predict(test_sample_x)

print("Result from grid seasrched decision tree")
print(np.sum(dt_grid_result == test_sample_y['Survived']))
print(np.sum(dt_grid_result == test_sample_y['Survived']) / len(result))

Result from grid seasrched decision tree
18
0.10112359550561797


In [172]:
# SVM
param_grid = [
        {
            'degree': [1, 2, 3, 4, 5, 6, 7],
            'gamma': [0.1, 1, 10]
        },
]

sv_grid_search = GridSearchCV(sv, param_grid, cv = 10,scoring = 'neg_mean_squared_error')

sv_grid_search.fit(sample_x, sample_y.values.ravel())

GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'degree': [1, 2, 3, 4, 5, 6, 7], 'gamma': [0.1, 1, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [173]:
sv_grid_search.best_params_

{'degree': 1, 'gamma': 0.1}

In [174]:
sv_grid_result = sv_grid_search.predict(test_sample_x)

print("Result from grid seasrched random forest")
print(np.sum(sv_grid_result==test_sample_y['Survived']))
print(np.sum(sv_grid_result == test_sample_y['Survived']) / len(result))

Result from grid seasrched random forest
127
0.7134831460674157


In [189]:
test_data = pd.read_csv("test.csv")
test_data.Age = test_data.Age.fillna(test_data.Age.mean())
test_data = test_data.replace('male', 0).replace('female', 1).replace('C', 0).replace('Q', 1).replace('S', 2)
test_data.Embarked = test_data.Embarked.fillna(2.0)
test_data.Fare = test_data.Fare.fillna(0) # to be dead

ids = test_data['PassengerId']
test_data.drop(['PassengerId',  'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,1
1,3,1,47.0,1,0,7.0000,2
2,2,0,62.0,0,0,9.6875,1
3,3,0,27.0,0,0,8.6625,2
4,3,1,22.0,1,1,12.2875,2


In [190]:
output = pd.DataFrame(columns=["PassengerId","Survived"])


for i in range(len(ids)) :
    row = test_data[i:i+1]
    predicted = rf_grid_search.predict(row)
    insert = pd.Series([ids[i], predicted[0]], index=output.columns)
    output = output.append(insert, ignore_index=True)

#    df = pd.DataFrame([[ids[i], predicted[0]]], columns=["PassengerId","Survived"])
#    output = output.append(df, ignore_index=True)
#print(output)

In [191]:
output.describe()

,PassengerId,Survived
count,418,418
unique,418,2
top,1023,0
freq,1,292


In [192]:
output.to_csv('4th_submission_by_grid_search_RandomForrest.csv', index=False)